In [ ]:
import ROOT
import time as t
from array import array
import sys
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib notebook
%matplotlib inline


In [ ]:
from dask.distributed import LocalCluster, Client

RDataFrame = ROOT.RDataFrame
DaskRDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame


def create_connection():
    """
    Setup connection to a Dask cluster. Two ingredients are needed:
    1. Creating a cluster object that represents computing resources. This can be
       done in various ways depending on the type of resources at disposal. To use
       only the local machine (e.g. your laptop), a `LocalCluster` object can be
       used. This step can be skipped if you have access to an existing Dask
       cluster; in that case, the cluster administrator should provide you with a
       URL to connect to the cluster in step 2. More options for cluster creation
       can be found in the Dask docs at
       http://distributed.dask.org/en/stable/api.html#cluster .
    2. Creating a Dask client object that connects to the cluster. This accepts
       directly the object previously created. In case the cluster was setup
       externally, you need to provide an endpoint URL to the client, e.g.
       'https://myscheduler.domain:8786'.

    Through Dask, you can connect to various types of cluster resources. For
    example, you can connect together a set of machines through SSH and use them
    to run your computations. This is done through the `SSHCluster` class. For
    example:

    ```python
    from dask.distributed import SSHCluster
    cluster = SSHCluster(
        # A list with machine host names, the first name will be used as
        # scheduler, following names will become workers.
        hosts=["machine1","machine2","machine3"],
        # A dictionary of options for each worker node, here we set the number
        # of cores to be used on each node.
        worker_options={"nprocs":4,},
    )
    ```

    Another common usecase is interfacing Dask to a batch system like HTCondor or
    Slurm. A separate package called dask-jobqueue (https://jobqueue.dask.org)
    extends the available Dask cluster classes to enable running Dask computations
    as batch jobs. In this case, the cluster object usually receives the parameters
    that would be written in the job description file. For example:

    ```python
    from dask_jobqueue import HTCondorCluster
    cluster = HTCondorCluster(
        cores=1,
        memory='2000MB',
        disk='1000MB',
    )
    # Use the scale method to send as many jobs as needed
    cluster.scale(4)
    ```

    In this tutorial, a cluster object is created for the local machine, using
    multiprocessing (processes=True) on 2 workers (n_workers=2) each using only
    1 core (threads_per_worker=1) and 2GiB of RAM (memory_limit="2GiB").
    """
    cluster = LocalCluster(n_workers=2, threads_per_worker=1, processes=True, memory_limit="4GiB")
    client = Client(cluster)
    return client

if __name__ == '__main__':

    connection = create_connection()

# Prepare file paths

In [ ]:
if __name__ == '__main__':

    #You can probably work with any one voltage, just comment out the others
    series= {"0V":["23220420_160544", "23220420_173224", "23220420_213243", "23220421_013301", "23220421_053320"],
             "20V":["23220417_012632","23220417_052651","23220417_092710","23220417_132730","23220417_172749","23220417_212807","23220418_012825","23220418_052843",
             "23220418_092901","23220418_132921","23220418_172940"],
             "50V":["23220421_161919","23220421_201938","23220422_001957","23220422_042016","23220422_082034","23220422_163644","23220422_203703","23220423_003722",
             "23220423_043740","23220423_083758","23220423_123817","23220423_163836","23220423_203855","23220424_003913","23220424_043931","23220424_083950",
             "23220424_124008","23220424_164027","23220424_204045","23220425_004105","23220425_161451","23220425_201511","23220426_001529","23220426_041547",
             "23220426_081605","23220426_181218","23220426_221238","23220427_021256","23220427_061315"]}
    voltages={"0V","20V","50V"} #Comment out voltages you're not using

    run_number = 24
    detNo = [5]
    detList={"5":"S101"}
    chanList={"S101":["PBS2","PFS2","PAS2","PT"]}
    chanHV={"S101":["PFS2","PCS2","PES2","PT"]}

    #Loading data
    # catalog = CDMSDataCatalog(default_fetchdir="/sdf/group/supercdms/data")

    ProdTag = "S101Ba"
    base = "/cvmfs/data/CDMS/RDataFrame_testing/data/s101/"
    file_list={}
    for v in voltages:
        fv=[]
        for s in series[v]:
            fv.append(f'{base}{ProdTag}_{s}.root')
        file_list[v]=fv

    fetchresult={}
    for v in voltages:
        print("Fetching {} files for ".format(len(file_list[v]))+v+" ...")
    #     fetchresult[v] = catalog.fetch(file_list[v])

In [ ]:
if __name__ == '__main__':

    files = file_list.values()
    files = [x for xs in files for x in xs]
    for i in files:
        print(i)
    #     !du -h {i}

# RDataFrame

In [ ]:
if __name__ == '__main__':

    chains_zip5 = {}
    chains_events = {}
    for bias in file_list:
        chains_zip5[bias] = ROOT.TChain()
        chains_events[bias] = ROOT.TChain()
        for file in file_list[bias]:
            chains_zip5[bias].Add(f'{file}?#rqDir/zip5')
            chains_events[bias].Add(f'{file}?#rqDir/eventTree')
        chains_zip5[bias].AddFriend(chains_events[bias])
    df = {}
    for bias in file_list:
        df[bias] = DaskRDataFrame(chains_zip5[bias], daskclient=connection)
        # df[bias] = RDataFrame(chains_zip5[bias])

In [ ]:
if __name__ == '__main__':

    print(df[bias].Count().GetValue())

## Cuts

In [ ]:
if __name__ == '__main__':

    ROOT.gInterpreter.Declare(
    '''
    bool cBaselineCut_0V(double PAS2bs) {
        double mu_gaus=32770.21147818;
        double sigma_gaus=85.51944755;
        return (PAS2bs > mu_gaus-sigma_gaus)&&(PAS2bs < mu_gaus+1.5*sigma_gaus);
    }
    '''
    )

    ROOT.gInterpreter.Declare(
    '''
    bool cBaselineCut_20V(double PFS2bs) {
        double mu_gaus=32437.83876944;
        double sigma_gaus=50.79565417;
        return (PFS2bs > mu_gaus-sigma_gaus) && (PFS2bs < mu_gaus+1.5*sigma_gaus);
    }
    '''
    )
    ROOT.gInterpreter.Declare(
    '''
    bool cBaselineCut_50V(double PAS2bs) {
        double mu_gaus=32678.36298014;
        double sigma_gaus=795.65838302;
        return (PAS2bs > mu_gaus-sigma_gaus) && (PAS2bs < mu_gaus+1.5*sigma_gaus);
    }
    '''
    )
    ROOT.gInterpreter.Declare(
    '''
    bool cBasicCut(int TriggerType, int TriggerDetectorNum) {
        return (TriggerType == 1) && (TriggerDetectorNum ==5);
    }
    '''
    )

    @ROOT.Numba.Declare(["float","float"], "bool")
    def cChisqCut_0V(PTOFchisq, PTOFamps):
        amp=np.array([-999999,0,5e-7,1e-6,1.5e-6,2e-6,2.5e-6,3e-6,3.5e-6,4e-6,4.5e-6,5e-6,5.5e-6,6e-6,6.5e-6,7e-6,7.5e-6,8e-6,8.5e-6,9e-6])
        chi=(np.array([-1e7,4e4,4e4,4e4,4.2e4,4.5e4,5e4,5.5e4,7e4,1e5,1.3e5,1.9e5,2.7e5,4e5,6e5,9e5,1.5e6,3e6,8e6,2.5e7]))
        arc = np.interp(PTOFamps,amp, chi)
        return (PTOFchisq < arc)
    t.sleep(0.1)
    @ROOT.Numba.Declare(["float","float"], "bool")
    def cChisqCut_20V(PTOFchisq, PTOFamps):
        amp=np.array([-999999,0,5e-7,1e-6,1.5e-6,2e-6,2.5e-6,3e-6,3.5e-6,4e-6,4.5e-6,5e-6,5.5e-6,6e-6,6.5e-6,7e-6,7.5e-6,8e-6,8.5e-6])
        chi=(np.array([-1e7,4e4,4e4,4e4,4.2e4,4.5e4,5e4,5.5e4,7e4,1e5,1.3e5,1.9e5,2.7e5,4e5,6e5,1e6,2.5e6,7e6,2e7])) 
        arc = np.interp(PTOFamps,amp, chi)
        return (PTOFchisq < arc)

    @ROOT.Numba.Declare(["float","float"], "bool")
    def cChisqCut_50V(PTOFchisq, PTOFamps):
        amp=np.array([-999999,0,5e-7,1e-6,1.5e-6,2e-6,2.5e-6,
                      3e-6,3.5e-6,4e-6,4.5e-6,5e-6,5.5e-6,6e-6,
                      6.5e-6,7e-6,7.5e-6,8e-6,8.5e-6,9e-6,9.5e-6])
        chi=(np.array([-1e7,3.9e4,3.9e4,3.9e4,4e4,4.2e4,4.5e4
                       ,5e4,6e4,7.5e4,1e5,1.3e5,1.8e5,2.8e5,
                       4e5,6.8e5,1.4e6,3e6,7e6,1.5e7,4e7]))
        arc = np.interp(PTOFamps,amp, chi)
        return (PTOFchisq < arc)


## Spectrum

In [ ]:
%%time

if __name__ == '__main__':


    filterd_df = {}
    for v in ['0V','20V','50V']:   
        filterd_df[v] = df[v].Filter(f'Numba::cChisqCut_{v}(PTOFchisq, PTOFamps)')\
                             .Filter('cBasicCut(TriggerType, TriggerDetectorNum)')\
                             .Filter(f'cBaselineCut_{v}(PAS2bs)')\
                             .Define('Int_sum','PBS2INTall+PFS2INTall+PAS2INTall')
    hists ={}
    for v in ['0V','20V','50V']:
        name = v
        hist_signiture = name,name,200,0,0.045
        hists[v] = filterd_df[v].Histo1D(hist_signiture,'Int_sum')

In [ ]:
if __name__ == '__main__':

    print(df[v].Filter('cBasicCut(TriggerType, TriggerDetectorNum)').Count().GetValue())

In [ ]:
if __name__ == '__main__':

    print(df[v].Filter(f'Numba::cChisqCut_{v}(PTOFchisq, PTOFamps)').Count().GetValue())

In [ ]:
if __name__ == '__main__':

    print(filterd_df[v].Count().GetValue())

In [ ]:
if __name__ == '__main__':

    def TH2python(hist):
        N = hist.GetNbinsX()
        centers = np.array([hist.GetBinCenter(i) for i in range(N)])
        values = np.array([hist.GetBinContent(i) for i in range(N)])
        return (centers,values)

In [ ]:
%%time

if __name__ == '__main__':


    fig,ax=plt.subplots()
    for v in ["0V","20V","50V"]:
        hist = hists[v].GetValue()
        plt.step(*TH2python(hist),label=v+'root')

    plt.yscale("log")
    #plt.ylim(0,1e-2)
    plt.xlabel("Integrated amplitude (a.u.)")
    plt.legend(loc=1)
    plt.grid(axis="both",which='major')
    #plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.title(r"${}^{133}$Ba spectrum")
    plt.text(0.5, 0.5, 'SuperCDMS work in progress', transform=ax.transAxes,fontsize=30, color='gray', alpha=1,ha='center', va='center')
    _ = plt.ylabel("Event rate (/ kg /day)")